<a href="https://colab.research.google.com/github/tommasomncttn/NLP-Disaster-Tweet-Detection/blob/main/ULMfiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from fastai.text.all import *
import pandas as pd
from sklearn.model_selection import train_test_split

# Fine tune a pre-trained model as a classifier

In [26]:
df = pd.read_csv("/content/drive/MyDrive/ML_proj/train.csv")
df.drop(columns = ["id", "keyword", "location"], inplace=True)

In [12]:
dls = TextDataLoaders.from_df(df, path='.', valid_pct=0.2, seed=None,
                          text_col=0, label_col=1, label_delim=None,
                          y_block=None, text_vocab=None, is_lm=False,
                          valid_col=None, tok_tfm=None,
                          tok_text_col='text', seq_len=72)
dls.show_batch()

In [14]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [15]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.590429,0.515331,0.758213,03:29


epoch,train_loss,valid_loss,accuracy,time
0,0.536059,0.444147,0.791064,08:57
1,0.496015,0.419287,0.819974,08:54
2,0.426667,0.404320,0.831143,08:55
3,0.367773,0.421032,0.819317,09:50


In [16]:
learn.show_results()

,text,category,category_
0,xxbos xxup info xxup s. xxup wnd : xxunk / 6 . xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup inst xxup apch . xxup rwy 05 . xxup curfew xxup in xxup oper xxup until 2030 xxup z. xxup taxiways xxup foxtrot 5 & & xxup foxtrot 6 xxup navbl . xxup tmp : 10 .,0,1
1,xxbos xxmaj truth … \n https : / / t.co / xxunk \n▁ # xxmaj news \n▁ # xxup bbc \n▁ # xxup cnn \n▁ # xxmaj islam \n▁ # xxmaj truth \n▁ # god \n▁ # xxup isis \n▁ # terrorism \n▁ # xxmaj quran \n▁ # xxmaj lies http : / / t.co / xxunk,0,1
2,xxbos # nowplaying * xxmaj cliff xxmaj richard - i xxmaj could xxmaj easily xxmaj fall ( in xxmaj love xxmaj with xxmaj you ) ( & & xxmaj xxunk ) * # xxmaj internet # xxmaj xxunk # xxmaj radio xxmaj on http : / / t.co / xxunk,0,0
3,xxbos i xxup hate xxup when xxup i m xxup trying xxup to xxup xxunk xxup my xxup hair xxup and xxup my xxup brother xxup comes xxup xxunk xxup in xxup behind xxup me xxup and xxup xxunk xxup me i xxup just xxup burned xxup my xxup finger,0,0
4,xxbos ' so again make sure to evacuate past the fire doors . xxmaj any questions ? xxmaj yes ? ' \n ' why would we open the doors to the fire ! ! ? ! ? ? ! ? ' \n\n i … i .. i ca nt,0,1
5,xxbos xxmaj haha xxmaj south xxmaj tampa is getting flooded xxunk xxup wait a xxup second i xxup live xxup in xxup south xxup tampa xxup what xxup am i xxup gon na xxup do xxup what xxup am i xxup gon na xxup do xxup xxunk # flooding,1,1
6,xxbos xxmaj baltimore xxmaj city : xxup xxunk xxup north xxup at xxup mp xxunk ( fort xxup xxunk xxup tunnel xxup bore 3 : xxmaj collision : xxup xxunk xxup north xxup at xxup mp xxunk ( fort xxup xxunk xxup tunnel xxup bore 3 xxmaj xxunk …,1,1
7,xxbos xxmaj xxunk xxmaj casualties xxmaj all xxmaj across xxmaj the xxmaj xxunk xxmaj war xxmaj being xxmaj xxunk xxmaj on xxmaj all xxmaj the xxmaj corners xxmaj of xxmaj the xxmaj planet [ on xxmaj all xxmaj levels ] http : / / t.co / xxup xxunk,1,1
8,xxbos xxrep 4 ? xxunk xxmaj xxunk ? ? xxrep 4 ? xxmaj will xxmaj you xxmaj still xxmaj love xxmaj me xxmaj tomorrow http : / / t.co / xxunk ? ? xxunk ? ? ? ? # xxmaj whirlwind http : / / t.co / xxunk,0,0


# Fine tune the LM on a disaster tweets dataset and then fine tune it as a classifier

In [ ]:
#TODO clean dataset

In [27]:
dataset = pd.read_csv("/content/drive/MyDrive/ML_proj/tweets.csv",)
dataset.drop(columns = ["id", "keyword",	"location", "target"], inplace =True)
dataset.head()

NameError: ignored

In [11]:
dls_lm = TextDataLoaders.from_df(dataset, path='.', valid_pct=0.2, seed=None,
                          text_col=0, label_col=1, label_delim=None,
                          y_block=None, text_vocab=None, is_lm=True,
                          valid_col=None, tok_tfm=None,
                          tok_text_col='text', seq_len=72, bs=32)
dls_lm.show_batch(max_n=5)

,text,text_
0,xxbos xxmaj one being carried out on a stretcher . xxbos xxup xxunk xxmaj xxunk xxmaj street ( xxunk ) ( all xxmaj directions ) at the junction of xxmaj xxunk xxmaj lane - xxmaj there is a collapsed xxunk cover at th … https : / / t.co / xxunk xxbos xxmaj police figures show far xxunk people in xxmaj australia have been charged with bushfire arson https : / /,xxmaj one being carried out on a stretcher . xxbos xxup xxunk xxmaj xxunk xxmaj street ( xxunk ) ( all xxmaj directions ) at the junction of xxmaj xxunk xxmaj lane - xxmaj there is a collapsed xxunk cover at th … https : / / t.co / xxunk xxbos xxmaj police figures show far xxunk people in xxmaj australia have been charged with bushfire arson https : / / t.co
1,"gym and gaming on their phone , but can never drown out the xxunk … xxbos xxmaj please pray for me i ai n’t tryna see a stretcher xxbos a rescue operation organized by a local family on xxmaj kangaroo xxmaj island transported injured koalas from areas devastated by bushfires . xxup t … xxbos parallel - xxunk # xxunk a whirlwind xxunk with a killer xxunk ! xxmaj to watch xxup","and gaming on their phone , but can never drown out the xxunk … xxbos xxmaj please pray for me i ai n’t tryna see a stretcher xxbos a rescue operation organized by a local family on xxmaj kangaroo xxmaj island transported injured koalas from areas devastated by bushfires . xxup t … xxbos parallel - xxunk # xxunk a whirlwind xxunk with a killer xxunk ! xxmaj to watch xxup click"
2,rape xxunk … https : / / t.co / xxunk xxbos xxmaj xxunk the eastern edge & & is in the direct drift of any spillage because of the strong xxmaj xxunk … xxbos xxmaj no xxunk today but our local factory is sadly still ablaze # xxunk https : / / t.co / xxunk xxbos xxmaj army ids xxmaj two xxmaj xxunk xxmaj killed by xxmaj roadside xxmaj bomb in xxmaj,xxunk … https : / / t.co / xxunk xxbos xxmaj xxunk the eastern edge & & is in the direct drift of any spillage because of the strong xxmaj xxunk … xxbos xxmaj no xxunk today but our local factory is sadly still ablaze # xxunk https : / / t.co / xxunk xxbos xxmaj army ids xxmaj two xxmaj xxunk xxmaj killed by xxmaj roadside xxmaj bomb in xxmaj afghanistan
3,"carried away xxbos — everything , even if it was n't his fault … ⠀ that was the reason he made the attempt to get up , … https : / / t.co / xxunk xxbos xxmaj yes ! 🤣 < xxunk inundation of childhood memories > i want some ! xxbos xxmaj wait this blew up xxunk te xxunk ? ? xxmaj wow https : / / t.co / xxunk xxbos","away xxbos — everything , even if it was n't his fault … ⠀ that was the reason he made the attempt to get up , … https : / / t.co / xxunk xxbos xxmaj yes ! 🤣 < xxunk inundation of childhood memories > i want some ! xxbos xxmaj wait this blew up xxunk te xxunk ? ? xxmaj wow https : / / t.co / xxunk xxbos xxmaj"
4,/ / t.co / xxunk xxbos xxmaj koala xxmaj who xxmaj burned xxmaj all 4 xxmaj paws xxmaj in xxmaj fires xxmaj is xxmaj so xxmaj happy xxmaj to xxmaj finally xxmaj be xxmaj safe https : / / t.co / xxunk xxbos xxmaj hi xxmaj xxunk . i do xxunk for the disruption . xxmaj due to a freight train derailment in the xxmaj hitchin area … https : / /,/ t.co / xxunk xxbos xxmaj koala xxmaj who xxmaj burned xxmaj all 4 xxmaj paws xxmaj in xxmaj fires xxmaj is xxmaj so xxmaj happy xxmaj to xxmaj finally xxmaj be xxmaj safe https : / / t.co / xxunk xxbos xxmaj hi xxmaj xxunk . i do xxunk for the disruption . xxmaj due to a freight train derailment in the xxmaj hitchin area … https : / / t.co


In [12]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [13]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.394942,3.934551,0.342181,51.139194,00:16


In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.956281,3.794227,0.354541,44.443886,00:17
1,3.783934,3.657912,0.371747,38.780285,00:17
2,3.625871,3.591972,0.379057,36.305614,00:18
3,3.443267,3.538791,0.387146,34.425289,00:17
4,3.265514,3.502216,0.393246,33.188927,00:18
5,3.117653,3.498070,0.397723,33.051594,00:19
6,2.984990,3.494832,0.398656,32.944763,00:17
7,2.881455,3.502855,0.401540,33.210125,00:17
8,2.804682,3.511593,0.401121,33.501598,00:17
9,2.772490,3.512924,0.400637,33.546204,00:17


In [21]:
learn.save_encoder('finetuned')

In [20]:
TEXT = "there was a fire"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

there was a fire in Bound Brook , Prompting National Emergency Services , in the South Pacific Area . All conditions had … https : / / t.co / T This
there was a fire in Coolock on Sunday just after the fire . But a storm could be expected to start … https : / / t.co / M Another moment when everyone gets inundated with condolences !


Fine tuning the classifier

In [32]:
dls_clas = TextDataLoaders.from_df(df, path='.', valid_pct=0.2, seed=None,
                          text_col=0, label_col=1, label_delim=None,
                          y_block=None, text_vocab=dls_lm.vocab, is_lm=False,
                          valid_col=None, tok_tfm=None,
                          tok_text_col='text', seq_len=72)
dls_clas.show_batch()

,text,category
0,xxbos _ xxunk xxrep 5 ? xxup retweet xxunk xxrep 7 ? xxunk xxrep 5 ? xxup follow xxup all xxup who xxup rt xxunk xxrep 7 ? xxunk xxrep 5 ? xxup xxunk xxunk xxrep 7 ? xxunk xxrep 5 ? xxup gain xxup with xxunk xxrep 7 ? xxunk xxrep 5 ? xxup follow ? xxunk # xxup xxunk xxunk # xxup xxunk,0
1,xxbos . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : xxup rt xxunk : # xxunk xxunk xxmaj indian xxmaj army xxunk _ http : / / t.co / xxunk g,0
2,xxbos xxup info xxup xxunk xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup xxunk xxup xxunk . xxup xxunk xxunk . xxup curfew xxup in xxup xxunk xxup until 2030 xxup xxunk xxup xxunk xxup xxunk 5 & & xxup xxunk 6 xxup xxunk . xxup xxunk : 10 . xxup xxunk : xxunk / 6 .,0
3,xxbos xxup info xxup s. xxup xxunk : xxunk / 6 . xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup xxunk xxup xxunk . xxup xxunk xxunk . xxup curfew xxup in xxup xxunk xxup until 2030 xxup xxunk xxup xxunk xxup xxunk 5 & & xxup xxunk 6 xxup xxunk . xxup xxunk : 10 .,0
4,xxbos i xxmaj hate xxmaj to xxmaj talking xxmaj xxunk xxmaj with xxmaj my xxmaj xxunk … i xxmaj mean i xxmaj love xxmaj her xxmaj as xxmaj to xxmaj death xxmaj but xxmaj she xxmaj talk xxmaj so xxmaj damn xxmaj much xxmaj xxunk xxrep 3 h xxrep 3 e xxunk xxrep 3 ! xxrep 6 ?,0
5,xxbos xxmaj truth … xxunk https : / / t.co / xxunk xxunk # xxmaj news xxunk # xxup bbc xxunk # xxup cnn xxunk # xxmaj islam xxunk # xxmaj truth xxunk # god xxunk # xxup isis xxunk # terrorism xxunk # xxmaj quran xxunk # xxmaj lies http : / / t.co / xxunk,1
6,xxbos xxmaj truth … xxunk https : / / t.co / xxunk xxunk # xxmaj news xxunk # xxup bbc xxunk # xxup cnn xxunk # xxmaj islam xxunk # xxmaj truth xxunk # god xxunk # xxup isis xxunk # terrorism xxunk # xxmaj quran xxunk # xxmaj lies http : / / t.co / xxunk,0
7,xxbos xxmaj no # news of # hostages in # xxmaj libya xxunk http : / / t.co / xxunk xxunk # xxmaj india # terrorism # xxmaj africa # xxup ap # xxup ts # xxup xxunk # xxmaj news # xxup xxunk # xxup xxunk # xxup bjp http : / / t.co / xxunk,1
8,xxbos xxmaj truth … xxunk https : / / t.co / xxunk xxunk # xxmaj news xxunk # xxup bbc xxunk # xxup cnn xxunk # xxmaj islam xxunk # xxmaj truth xxunk # god xxunk # xxup isis xxunk # terrorism xxunk # xxmaj quran xxunk # xxmaj lies http : / / t.co / xxunk,1


In [33]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [34]:
learn = learn.load_encoder('finetuned')

In [35]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.540243,0.479562,0.780552,00:09


In [36]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.520950,0.473163,0.786465,00:10


In [37]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.487397,0.454189,0.795007,00:11


In [39]:
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.364108,0.468768,0.806176,00:11
1,0.342386,0.478863,0.800920,00:11
2,0.296341,0.512496,0.797635,00:11
3,0.264607,0.516879,0.800920,00:11
